In [31]:
from py2neo import Graph
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ipyleaflet import Map,CircleMarker,FullScreenControl,basemaps, basemap_to_tiles
import geopandas
import descartes
import matplotlib

Connect to Neo4J using default userid and modified

In [32]:
graph = Graph("bolt://localhost:7687", auth=("neo4j", "test"))

Use the city object to centre maps. Leaflet will be used to plot the maps within the notebook, although the CSVs are exported to allow the final maps to be created in other tooling.

In [33]:
df_map = graph.run(
  """MATCH (c:City) RETURN c.location.longitude as long, c.location.latitude as lat"""
).to_data_frame()

Utility Functions

In [38]:
def makeMap() : 
    """ Make a Map based on the centre retrieved from Neo4J
    """
    # Use .item() as an error occurs trying to use numpy value types. item() converts to native Python
    m = Map(center=[df_map['lat'].item(),df_map['long'].item()],zoom=10)
    dark_matter_layer = basemap_to_tiles(basemaps.CartoDB.DarkMatter)
    m.add_layer(dark_matter_layer)
    m.add_control(FullScreenControl())
    return m

def getColorFor2008To2018Change(value) : 
    """  Return a colour based on a value
    """
    if(value <= 100) :
        return 'red'
    elif (value > 101 and value < 600)  :
        return 'orange'
    else :
        return 'green'  

def getColourForFlag(value) : 
    """  Return a colour based on a flag
    """
    if(value == 'U') :
        return "Green"
    else :
        return "Red"    
    
    
def createMarkers(x,m,field, size, colorFunction) :
    circle_marker = CircleMarker()
    circle_marker.location = (x['latitude'],x['longitude'])
    circle_marker.radius = size
    circle_marker.color = "red"
    circle_marker.stroke = False
    circle_marker.fill_opacity = 1
    circle_marker.fill_color = colorFunction(x[field])
    m.add_layer(circle_marker)
def addMarkers(m,df,field, size, colorFunction) :
    df.apply(lambda x: createMarkers(x,m,field, size, colorFunction), axis=1)
    #for index, row in df.iterrows():
       
        
def updateDataFrame(df, suffix) :
    """
        Update the DataFrame to include changes in average between years
    """
    # need to set the type of avgXXXX and add a suffix
    df['avg1998' + suffix] = df['avg1998'].astype(np.int64)
    df['avg2008' + suffix] = df['avg2008'].astype(np.int64)
    df['avg2018' + suffix] = df['avg2018'].astype(np.int64)
    
    # calculate the change between years and add it to the dataframe
    df["change1998to2018" + suffix] = df.apply(lambda x: change1998to2018(x, suffix), axis=1)
    df["change1998to2008" + suffix] = df.apply(lambda x: change1998to2008(x, suffix), axis=1)
    df["change2008to2018" + suffix] = df.apply(lambda x: change2008to2018(x, suffix), axis=1)

def change1998to2018(x,suffix):
  return changeBetweenYears(x,'avg1998' + suffix,'avg2018' + suffix)
def change1998to2008(x,suffix):
  return changeBetweenYears(x,'avg1998' + suffix,'avg2008' + suffix)
def change2008to2018(x,suffix):
  return changeBetweenYears(x,'avg2008' + suffix,'avg2018' + suffix)

def changeBetweenYears(x,year1, year2) :
  if(x[year1] == 0 or x[year2] == 0) :
    return 0
  else :
    return (x[year2] - x[year1])/x[year1] * 100

def calcuteIncreaseDecreaseFor500And1000(x) : 
    """
        Calculates 
    """ 
    if((x['change1998to2018_500'] > x['change1998to2018_1000']) and percentChangeGreaterThan5Percent(x['change1998to2018_500'], x['change1998to2018_1000'])) :
        return 'U'
    elif((x['change1998to2018_500'] < x['change1998to2018_1000']) and percentChangeGreaterThan5Percent(x['change1998to2018_1000'], x['change1998to2018_500'])) :
        return 'D'
    else:
        return 'N'
def percentChangeGreaterThan5Percent(val1, val2) :
    if(val2 == 0) :
        return False
    if(((val1 - val2) / val2) > .05) :
        return True
    else :
        return False
    
def writeDataFrameToCSV(dataframe, filename) :
    dataframe.to_csv("../csvs/" + filename, index=False);

Create a CSV for all the station locations with lines. 

In [ ]:
df_stations = graph.run(
    """MATCH (l:Line)-[:IS_ON]-(s)
    RETURN l.name as lineName, s.name as stationName, s.location.x as longitude, s.location.y as latitude
    """).to_data_frame();
writeDataFrameToCSV(df_stations, "lines_and_stations.csv");    
    

Create a Map that shows whether the change in sales prices is greater within 500m or between 1000m and 1500m. A field is added that has 'U' for 500 > 1000 'D' For 500 < 1000

In [41]:
df_500 = graph.run(
  """MATCH (l:Line)<-[i:IS_ON]-(s)<-[:IS_WITHIN {distance : 500}]-(p)<-[t:SALE_IN]-(sl)
  WITH DISTINCT(sl) as sl,s,l 
  WITH  [x IN collect(sl) WHERE x.year = 1998 | x.price] as sales1998,[x IN collect(sl) WHERE x.year = 2008 | x.price] as sales2008,[x IN collect(sl) WHERE x.year = 2018 | x.price] as sales2018,s,l
  RETURN l.name as lineName, s.name as stationName, s.location.x as longitude, s.location.y as latitude, COALESCE(apoc.coll.avg(sales1998),0) as avg1998, COALESCE(apoc.coll.avg(sales2008),0) as avg2008,COALESCE(apoc.coll.avg(sales2018),0) as avg2018"""
).to_data_frame()

df_1000 = graph.run(
  """MATCH (l:Line)<-[i:IS_ON]-(s)<-[:IS_WITHIN {distance : 1000}]-(p)<-[t:SALE_IN]-(sl)
  WITH DISTINCT(sl) as sl,s,l 
  WITH  [x IN collect(sl) WHERE x.year = 1998 | x.price] as sales1998,[x IN collect(sl) WHERE x.year = 2008 | x.price] as sales2008,[x IN collect(sl) WHERE x.year = 2018 | x.price] as sales2018,s,l
  RETURN l.name as lineName, s.name as stationName, s.location.x as longitude, s.location.y as latitude, COALESCE(apoc.coll.avg(sales1998),0) as avg1998, COALESCE(apoc.coll.avg(sales2008),0) as avg2008,COALESCE(apoc.coll.avg(sales2018),0) as avg2018"""
).to_data_frame()

In [42]:
updateDataFrame(df_500,'_500')
updateDataFrame(df_1000,'_1000')

df_combined = pd.merge(df_500, df_1000, on=['lineName', 'stationName','latitude','longitude'])
df_combined["change"] = df_combined.apply(lambda x: calcuteIncreaseDecreaseFor500And1000(x), axis=1)

In [43]:
m1 = makeMap()
m1

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [44]:
addMarkers(m1,df_combined,'change',6, getColourForFlag)


In [45]:
writeDataFrameToCSV(df_combined,"increase_decrease_between_500_and_1000_1998_to_2018.csv")

In [47]:
def runQueryAndUpdateMap (m) : 
    df_500 = graph.run(
      """MATCH (s:Station)<-[:IS_WITHIN {distance : 500}]-(p)<-[t:SALE_IN]-(sl)
      WITH DISTINCT(sl) as sl,s
      WITH  [x IN collect(sl) WHERE x.year = 1998 | x.price] as sales1998,[x IN collect(sl) WHERE x.year = 2008 | x.price] as sales2008,[x IN collect(sl) WHERE x.year = 2018 | x.price] as sales2018,s
      RETURN s.name as stationName, s.location.x as longitude, s.location.y as latitude, COALESCE(apoc.coll.avg(sales1998),0) as avg1998, COALESCE(apoc.coll.avg(sales2008),0) as avg2008,COALESCE(apoc.coll.avg(sales2018),0) as avg2018"""
    ).to_data_frame()
   
    df_1000 = graph.run(
      """MATCH (s:Station)<-[:IS_WITHIN {distance : 1000}]-(p)<-[t:SALE_IN]-(sl)
      WITH DISTINCT(sl) as sl,s
      WITH  [x IN collect(sl) WHERE x.year = 1998 | x.price] as sales1998,[x IN collect(sl) WHERE x.year = 2008 | x.price] as sales2008,[x IN collect(sl) WHERE x.year = 2018 | x.price] as sales2018,s
      RETURN s.name as stationName, s.location.x as longitude, s.location.y as latitude, COALESCE(apoc.coll.avg(sales1998),0) as avg1998, COALESCE(apoc.coll.avg(sales2008),0) as avg2008,COALESCE(apoc.coll.avg(sales2018),0) as avg2018"""
    ).to_data_frame()

    updateDataFrame(df_500, '_500')
    updateDataFrame(df_1000, '_1000')

    addMarkers(m,df_1000,'change1998to2018_1000',5,getColorFor2008To2018Change)
    addMarkers(m,df_500,'change1998to2018_500',3,getColorFor2008To2018Change)
    
    df_combined = pd.merge(df_500, df_1000, on=['stationName','latitude','longitude'])
    df_combined["change"] = df_combined.apply(lambda x: calcuteIncreaseDecreaseFor500And1000(x), axis=1)
    
    return df_combined

In [48]:
m2 = makeMap()
df_combined = runQueryAndUpdateMap(m2)
writeDataFrameToCSV(df_combined,"differences_between_500_and_1000_for_all_building_type.csv")
m2

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [13]:
print(df_combined['change'].value_counts())

U    276
D    267
N     94
Name: change, dtype: int64


In [14]:
addMarkers(m2,df_1000,'change2008to2018',5,getColorFor2008To2018Change)
addMarkers(m2,df_500,'change2008to2018',3,getColorFor2008To2018Change)

In [15]:
def runDistanceQuery(lower, upper, postcodeDistance) :
    
    params = {
        "lower": lower,
        "upper": upper,
        "postcodeDistance" : postcodeDistance
    }
    
    df = graph.run(
    """MATCH (c:City)
    WITH c
    MATCH (s:Station) WHERE DISTANCE(s.location, c.location) >= $lower AND DISTANCE(s.location, c.location) < $upper
    WITH s
    MATCH (s)<-[:IS_WITHIN {distance : $postcodeDistance}]-(p)<-[t:SALE_IN]-(sl)
    WITH DISTINCT(sl) as sl,s
    WITH  [x IN collect(sl) WHERE x.year = 1998 | x.price] as sales1998,[x IN collect(sl) WHERE x.year = 2008 | x.price] as sales2008,[x IN collect(sl) WHERE x.year = 2018 | x.price] as sales2018,s
    RETURN s.name as stationName, s.location.x as longitude, s.location.y as latitude, COALESCE(apoc.coll.avg(sales1998),0) as avg1998, COALESCE(apoc.coll.avg(sales2008),0) as avg2008,COALESCE(apoc.coll.avg(sales2018),0) as avg2018""",params).to_data_frame()
    return df

In [16]:
df5_500 = runDistanceQuery(5000,6000,500)
df10_500 = runDistanceQuery(10000,11000,500)
df15_500 = runDistanceQuery(15000,16000,500)
df20_500 = runDistanceQuery(20000,21000,500)

df5_1000 = runDistanceQuery(5000,6000,1000)
df10_1000 = runDistanceQuery(10000,11000,1000)
df15_1000 = runDistanceQuery(15000,16000,1000)
df20_1000 = runDistanceQuery(20000,21000,1000)

In [17]:
updateDataFrame(df5_500, '')
updateDataFrame(df10_500, '')
updateDataFrame(df15_500, '')
updateDataFrame(df20_500, '')
updateDataFrame(df5_1000, '')
updateDataFrame(df10_1000, '')
updateDataFrame(df15_1000, '')
updateDataFrame(df20_1000, '')

In [18]:
m3 = makeMap()
m3

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [58]:
addMarkers(m3,df5_1000,'change2008to2018',6, getColorFor2008To2018Change)
writeDataFrameToCSV(df5_1000, "distance_5_range_1000.csv");
addMarkers(m3,df10_1000,'change2008to2018',6, getColorFor2008To2018Change)
writeDataFrameToCSV(df10_1000, "distance_10_range_1000.csv");
addMarkers(m3,df15_1000,'change2008to2018',6, getColorFor2008To2018Change)
writeDataFrameToCSV(df15_1000, "distance_15_range_1000.csv");
addMarkers(m3,df20_1000,'change2008to2018',6, getColorFor2008To2018Change)
writeDataFrameToCSV(df20_1000, "distance_20_range_1000.csv");

addMarkers(m3,df5_500,'change2008to2018',3, getColorFor2008To2018Change)
writeDataFrameToCSV(df5_500, "distance_5_range_500.csv");
addMarkers(m3,df10_500,'change2008to2018',3, getColorFor2008To2018Change)
writeDataFrameToCSV(df10_500, "distance_10_range_500.csv");
addMarkers(m3,df15_500,'change2008to2018',3, getColorFor2008To2018Change)
writeDataFrameToCSV(df15_500, "distance_15_range_500.csv");
addMarkers(m3,df20_500,'change2008to2018',3, getColorFor2008To2018Change)
writeDataFrameToCSV(df20_500, "distance_20_range_500.csv");

Look at Flats only for 500 and 1000

In [49]:
def runQueryAndUpdateMapForBuldingType (building_type, m) : 
    df_500 = graph.run(
      """MATCH (s:Station)<-[:IS_WITHIN {distance : 500}]-(p)<-[t:SALE_IN]-(sl {type : {t}})
      WITH DISTINCT(sl) as sl,s
      WITH  [x IN collect(sl) WHERE x.year = 1998 | x.price] as sales1998,[x IN collect(sl) WHERE x.year = 2008 | x.price] as sales2008,[x IN collect(sl) WHERE x.year = 2018 | x.price] as sales2018,s
      RETURN s.name as stationName, s.location.x as longitude, s.location.y as latitude, COALESCE(apoc.coll.avg(sales1998),0) as avg1998, COALESCE(apoc.coll.avg(sales2008),0) as avg2008,COALESCE(apoc.coll.avg(sales2018),0) as avg2018"""
    , t = building_type).to_data_frame()
   
    df_1000 = graph.run(
      """MATCH (s:Station)<-[:IS_WITHIN {distance : 1000}]-(p)<-[t:SALE_IN]-(sl {type : {t}})
      WITH DISTINCT(sl) as sl,s
      WITH  [x IN collect(sl) WHERE x.year = 1998 | x.price] as sales1998,[x IN collect(sl) WHERE x.year = 2008 | x.price] as sales2008,[x IN collect(sl) WHERE x.year = 2018 | x.price] as sales2018,s
      RETURN s.name as stationName, s.location.x as longitude, s.location.y as latitude, COALESCE(apoc.coll.avg(sales1998),0) as avg1998, COALESCE(apoc.coll.avg(sales2008),0) as avg2008,COALESCE(apoc.coll.avg(sales2018),0) as avg2018"""
    , t = building_type).to_data_frame()

    updateDataFrame(df_500, '_500')
    updateDataFrame(df_1000, '_1000')

    addMarkers(m,df_1000,'change1998to2018_1000',5,getColorFor2008To2018Change)
    addMarkers(m,df_500,'change1998to2018_500',3,getColorFor2008To2018Change)
    
    df_combined = pd.merge(df_500, df_1000, on=['stationName','latitude','longitude'])
    df_combined["change"] = df_combined.apply(lambda x: calcuteIncreaseDecreaseFor500And1000(x), axis=1)
    
    return df_combined


In [50]:
m4 = makeMap()
df_combined = runQueryAndUpdateMapForBuldingType("F",m4)
writeDataFrameToCSV(df_combined,"differences_between_500_and_1000_for_flats.csv")
m4


Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [22]:
print(df_combined['change'].value_counts())

U    266
D    257
N    108
Name: change, dtype: int64


In [51]:
m5 = makeMap()
df_combined = runQueryAndUpdateMapForBuldingType("S",m5)
writeDataFrameToCSV(df_combined,"differences_between_500_and_1000_for_semi.csv")
m5

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [25]:
print(df_combined['change'].value_counts())

N    205
U    204
D    187
Name: change, dtype: int64


In [54]:
m7 = makeMap()
df_combined = runQueryAndUpdateMapForBuldingType("D",m7)
writeDataFrameToCSV(df_combined,"differences_between_500_and_1000_for_detached.csv")
m7

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [27]:
print(df_combined['change'].value_counts())

N    264
U    154
D    105
Name: change, dtype: int64


In [56]:
m6 = makeMap()
df_combined = runQueryAndUpdateMapForBuldingType("T",m6)
writeDataFrameToCSV(df_combined,"differences_between_500_and_1000_for_terraced.csv")
m6

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [29]:
print(df_combined['change'].value_counts())

U    268
D    214
N    137
Name: change, dtype: int64


USE PAGERANK ALGORITHM TO FIND INTERESTING STATIONS

In [74]:
def runPageRank(m, direction) :
    df_500 = graph.run(
      """CALL algo.pageRank.stream('Station', 'LINE', {iterations:20, dampingFactor:0.85})
    YIELD nodeId, score
    WITH  algo.getNodeById(nodeId) AS s, score
    MATCH (s)<-[:IS_WITHIN {distance : 500}]-(p)<-[t:SALE_IN]-(sl)
    WITH DISTINCT(sl) as sl,s,score
    WITH  [x IN collect(sl) WHERE x.year = 1998 | x.price] as sales1998,[x IN collect(sl) WHERE x.year = 2008 | x.price] as sales2008,[x IN collect(sl) WHERE x.year = 2018 | x.price] as sales2018,s,score
    RETURN score, s.name as stationName, s.location.x as longitude, s.location.y as latitude, COALESCE(apoc.coll.avg(sales1998),0) as avg1998, COALESCE(apoc.coll.avg(sales2008),0) as avg2008,COALESCE(apoc.coll.avg(sales2018),0) as avg2018
    ORDER BY score """ + direction + """ 
    LIMIT 30""").to_data_frame();


    df_1000 = graph.run(
      """CALL algo.pageRank.stream('Station', 'LINE', {iterations:20, dampingFactor:0.85})
    YIELD nodeId, score
    WITH  algo.getNodeById(nodeId) AS s, score
    MATCH (s)<-[:IS_WITHIN {distance : 1000}]-(p)<-[t:SALE_IN]-(sl)
    WITH DISTINCT(sl) as sl,s,score
    WITH  [x IN collect(sl) WHERE x.year = 1998 | x.price] as sales1998,[x IN collect(sl) WHERE x.year = 2008 | x.price] as sales2008,[x IN collect(sl) WHERE x.year = 2018 | x.price] as sales2018,s,score
    RETURN score, s.name as stationName, s.location.x as longitude, s.location.y as latitude, COALESCE(apoc.coll.avg(sales1998),0) as avg1998, COALESCE(apoc.coll.avg(sales2008),0) as avg2008,COALESCE(apoc.coll.avg(sales2018),0) as avg2018
    ORDER BY score """ + direction + """ 
    LIMIT 30""").to_data_frame();
    

    updateDataFrame(df_500, '_500')
    updateDataFrame(df_1000, '_1000')

    addMarkers(m,df_1000,'change1998to2018_1000',5,getColorFor2008To2018Change)
    addMarkers(m,df_500,'change1998to2018_500',3,getColorFor2008To2018Change)

    writeDataFrameToCSV(df_1000,"page_rank_1000_" + direction + ".csv")
    writeDataFrameToCSV(df_500,"page_rank_500_" + direction + ".csv")

    

In [75]:
m8 = makeMap()
runPageRank(m8,"DESC")
m8

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [76]:
m9 = makeMap()
runPageRank(m9,"ASC")
m9

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …